# UHI Data Download and Processing Pipeline

## 1. Setup and Dependencies

In [1]:
import sys
import os
import pandas as pd
import numpy as np
import json
import matplotlib.pyplot as plt
from pathlib import Path
from datetime import datetime
import logging
import requests
import os
from tqdm import tqdm
import subprocess
import zipfile # For more robust unzipping
import time

# Imports for tile index processing
import geopandas as gpd
from shapely.geometry import box

# Add the project root to the Python path to allow importing from src
project_root = Path(os.getcwd()).parent  # Assumes notebook is in 'notebooks' subdir
sys.path.insert(0, str(project_root))

# Setup logging
logging.basicConfig(level=logging.INFO, format='%(asctime)s - %(levelname)s - %(message)s')

In [3]:

# --- Configuration for Data Download ---

# Configure parameters for the target city (e.g., NYC)
city_name = "NYC"

# Time window matching original notebooks (adjust if needed)
sentinel_time_window = "2021-06-01/2021-09-01"
lst_time_window = "2021-06-01/2021-09-01"

# Input files and general settings
data_dir = Path("data")
abs_output_dir = project_root / data_dir
uhi_csv = data_dir / city_name / "uhi.csv" # Path to UHI data
abs_uhi_csv = project_root / uhi_csv
# bbox_csv is no longer needed for bounds calculation

if not abs_uhi_csv.exists():
    raise FileNotFoundError(f"UHI data CSV not found at {abs_uhi_csv}. Cannot derive bounds.")
print(f"Loading bounds from UHI data: {abs_uhi_csv}")
uhi_df = pd.read_csv(abs_uhi_csv)
# Check if required columns exist
required_cols = ['Longitude', 'Latitude']
if not all(col in uhi_df.columns for col in required_cols):
     raise ValueError(f"UHI CSV must contain columns: {required_cols}")

# Load bounds
bounds = [
    uhi_df['Longitude'].min(),
    uhi_df['Latitude'].min(),
    uhi_df['Longitude'].max(),
    uhi_df['Latitude'].max()
]

# Load observation da
first_datetime_obj = pd.to_datetime(uhi_df['datetime'].iloc[0], format='%d-%m-%Y %H:%M')
# Format the date object into 'YYYY-MM-DD' string format
uhi_date_str = first_datetime_obj.strftime('%Y-%m-%d')
print(f"Representative UHI date (from first row): {uhi_date_str}")

Loading bounds from UHI data: /Users/arnav/MLC-Project/data/NYC/uhi.csv
Representative UHI date (from first row): 2021-07-24


## 2. Download Satellite Data for Cities

Now we'll download satellite imagery data (Sentinel-2 median composites, Landsat LST medians) for specific cities and time periods derived from the UHI data timestamps. Data is saved locally for use by the dataloader.

In [ ]:
# Cell from notebooks/download_data.ipynb (modified)

# Import functions
from src.ingest.get_median import create_and_save_cloudless_mosaic
# Import the modified LST download function
from src.ingest.create_sat_tensor_files import download_single_lst_median
import pandas as pd
from pathlib import Path
import os

# Parameters for Cloudless Mosaic (matching Sentinel2_GeoTIFF.ipynb)
mosaic_bands = ["B02", "B03", "B04", "B08"] # RGB+NIR for Clay compatibility
mosaic_resolution_m = 10
mosaic_cloud_cover = 30

# Parameters for LST Median (matching Landsat_LST.ipynb)
include_lst = False         # Whether to download LST
lst_resolution_m = 30      # Native resolution for Landsat LST

# Generate output path for the mosaic based on the new time window
start_dt_str = sentinel_time_window.split('/')[0].replace('-','')
end_dt_str = sentinel_time_window.split('/')[1].replace('-','')
band_str = "_".join(mosaic_bands)
cloudless_mosaic_filename = f"sentinel_{city_name}_{start_dt_str}_to_{end_dt_str}_cloudless_mosaic.npy"
cloudless_mosaic_path = abs_output_dir / city_name / "sat_files" / cloudless_mosaic_filename
# --- Verification ---
print(f"City: {city_name}")
print(f"Sentinel-2 Time Window: {sentinel_time_window}")
print(f"Sentinel-2 Cloud Cover Threshold: {mosaic_cloud_cover}%")
print(f"LST Time Window: {lst_time_window}")
print(f"Bounds derived from {uhi_csv.name}: {bounds}")
print(f"Target mosaic output path: {cloudless_mosaic_path}")
print(f"Include LST: {include_lst}")

Loading bounds from UHI data: /home/jupyter/UHI/MLC-Project/data/NYC/uhi.csv
City: NYC
Sentinel-2 Time Window: 2021-06-01/2021-09-01
Sentinel-2 Cloud Cover Threshold: 30%
LST Time Window: 2021-06-01/2021-09-01
Bounds derived from uhi.csv: [np.float64(-73.99445667), np.float64(40.75879167), np.float64(-73.87945833), np.float64(40.85949667)]
Target mosaic output path: /home/jupyter/UHI/MLC-Project/data/NYC/sat_files/sentinel_NYC_20210601_to_20210901_cloudless_mosaic.npy
Include LST: False


### 3. Download + Build DEM / DSM for NYC (AOI‑bounded, 1 ft grid)

1. **Tile‑index fetch**  
   * Pull both tile‑index shapefiles (DEM & LAS) from the NYS topo‑bathymetric 2017 FTP mirror.  
   * They live in `/BE_DEM/…zip` and `/LAS/…zip`.  
   * They’re unzipped to a temp folder, read by GeoPandas, then deleted.

2. **Intersect AOI**  
   * Your AOI bounds (given in decimal lat/long) are re‑projected to EPSG 2263.  
   * We select only the DEM `.tif` tiles and LAS `.laz` tiles whose polygons hit that AOI.

3. **Download source tiles**  
   * DEM tiles (`be_NYC_###.tif`) stream straight from FTP to `dem_tiles/`.  
   * LAS tiles (`hh_NYC_###.laz`) stream to `las_tiles/`.  
   * Progress bars show raw byte count; no FTP `SIZE` calls (the server blocks those).

4. **Build rasters**  
   * **DEM** – `gdal_merge.py` mosaics the few BE tiles → `dem_merged_epsg2263.tif`.  
   * **DSM** – PDAL crops the LAS hits, bins highest return (`output_type=max`), writes `dsm_epsg2263.tif` at 0.3048 m (1 ft) resolution.  
   * Both rasters are then re‑projected with `gdalwarp` to EPSG 4326:  
     `dem_epsg4326.tif`, `dsm_epsg4326.tif`.

5. **Clean‑up**  
   * All temp folders (`indices/`, `dem_tiles/`, `las_tiles/`) and intermediate rasters in EPSG 2263 are removed once the 4326 GeoTIFFs are verified non‑empty.  
   * Leftover artefacts = **zero**. Only the two final products remain.

**Data Source**

* Source LiDAR survey: *NYC Topobathymetric LiDAR 2017* FTP server.
* DEM tiles: bare‑earth (`be_…`) rasters published by NYS GIS Clearinghouse.  
* DSM: freshly generated from the raw point cloud because no public HH DSM tiles exist, although the
  state did publish them in 2017, they seem to have been deleted since.

**Outputs**

| File | CRS / Units | Resolution | Description |
|------|-------------|------------|-------------|
| `dem_epsg4326.tif` | EPSG 4326, metres in Z | ≈0.00000274° (~0.3048 m) | Bare‑earth ground elevation |
| `dsm_epsg4326.tif` | EPSG 4326, metres in Z | same grid | Surface elevation (roofs, canopy) |

In [4]:
# ─────────────────────────────────────────────────────────────────────────────
# CONFIG
# ─────────────────────────────────────────────────────────────────────────────
FTP_HOST       = "ftp.gis.ny.gov"
FTP_ROOT       = "/elevation/LIDAR/NYC_TopoBathymetric2017"

DEM_FTP_DIR    = f"{FTP_ROOT}/BE_DEM"     # be_NYC_###.tif + index zip
LAS_FTP_DIR    = f"{FTP_ROOT}/LAS"        # hh_NYC_###.laz + index zip
DEM_INDEX_FTP  = f"{DEM_FTP_DIR}/tileindex_NYC_topobathy_BE_DEM_2017.zip"
LAS_INDEX_FTP  = f"{LAS_FTP_DIR}/tileindex_2017_las.zip"

CRS_WGS84      = "EPSG:4326"
CRS_NAD83_NY   = "EPSG:2263"

root_out       = abs_output_dir / city_name / "elevation"
index_dir      = root_out / "indices"
dem_dir        = root_out / "dem_tiles"
las_dir        = root_out / "las_tiles"
root_out.mkdir(parents=True, exist_ok=True)
for d in (index_dir, dem_dir, las_dir):
    d.mkdir(parents=True, exist_ok=True)

dem_index_zip  = index_dir / "dem_index.zip"
las_index_zip  = index_dir / "las_index.zip"
merged_dem2263 = root_out / "dem_merged_epsg2263.tif"
dsm2263        = root_out / "dsm_epsg2263.tif"
dem4326        = root_out / "dem_epsg4326.tif"
dsm4326        = root_out / "dsm_epsg4326.tif"
pdal_json      = root_out / "dsm_pipeline.json"

logging.basicConfig(level=logging.INFO, format="%(levelname)s: %(message)s")

# ─────────────────────────────────────────────────────────────────────────────
# FTP helper (no SIZE)
# ─────────────────────────────────────────────────────────────────────────────
def ftp_fetch(remote: str, local: Path, blk: int = 1 << 20) -> bool:
    if local.exists():
        return True
    try:
        with ftplib.FTP(FTP_HOST) as ftp:
            ftp.login(); ftp.voidcmd("TYPE I")
            with open(local, "wb") as f, tqdm(desc=local.name, unit="iB", unit_scale=True) as bar:
                ftp.retrbinary(f"RETR {remote}",
                               lambda ch: (f.write(ch), bar.update(len(ch)))[-1],
                               blocksize=blk)
        if local.stat().st_size == 0:
            local.unlink(missing_ok=True)
            raise RuntimeError("zero‑byte file")
        return True
    except Exception as e:
        logging.error(f"FTP error {remote}: {e}")
        local.unlink(missing_ok=True)
        return False

def unzip(zip_path: Path, out_dir: Path) -> Path:
    with zipfile.ZipFile(zip_path) as z: z.extractall(out_dir)
    return next(out_dir.glob("*.shp"))

# ─────────────────────────────────────────────────────────────────────────────
# 1. Download and parse DEM tile‑index
# ─────────────────────────────────────────────────────────────────────────────
ftp_fetch(DEM_INDEX_FTP, dem_index_zip)
dem_index_shp = unzip(dem_index_zip, index_dir / "dem_index")
dem_index_gdf = gpd.read_file(dem_index_shp)
dem_index_gdf.crs = dem_index_gdf.crs or CRS_NAD83_NY

# AOI in index CRS
aoi = box(*bounds)
aoi_native = gpd.GeoSeries([aoi], crs=CRS_WGS84).to_crs(dem_index_gdf.crs).iloc[0]

dem_hits = dem_index_gdf[dem_index_gdf.geometry.intersects(aoi_native)]
for loc in dem_hits["location"]:
    fname = Path(loc).name
    ftp_fetch(f"{DEM_FTP_DIR}/{fname}", dem_dir / fname)

# merge DEM tiles
if not merged_dem2263.exists():
    srcs = [rasterio.open(p) for p in dem_dir.glob("*.tif")]
    out, trans = rio_merge(srcs)
    meta = srcs[0].meta.copy(); meta.update(driver="GTiff",
                                            height=out.shape[1],
                                            width=out.shape[2],
                                            transform=trans,
                                            nodata=-9999)
    with rasterio.open(merged_dem2263, "w", **meta) as dst: dst.write(out)
    for s in srcs: s.close()

# reproject DEM to EPSG:4326
if not dem4326.exists():
    subprocess.run(["gdalwarp", "-t_srs", CRS_WGS84, "-r", "bilinear",
                    "-dstnodata", "-9999", "-overwrite",
                    merged_dem2263, dem4326], check=True)

# ─────────────────────────────────────────────────────────────────────────────
# 2. Download LAS tile‑index and intersect AOI
# ─────────────────────────────────────────────────────────────────────────────
ftp_fetch(LAS_INDEX_FTP, las_index_zip)
las_index_shp = unzip(las_index_zip, index_dir / "las_index")
las_index_gdf = gpd.read_file(las_index_shp)
las_index_gdf.crs = las_index_gdf.crs or CRS_NAD83_NY

las_hits = las_index_gdf[las_index_gdf.geometry.intersects(aoi_native)]
for loc in las_hits["location"]:
    fname = Path(loc).name
    ftp_fetch(f"{LAS_FTP_DIR}/{fname}.laz", las_dir / f"{fname}.laz")

# ─────────────────────────────────────────────────────────────────────────────
# 3. Build DSM via PDAL (highest return)
# ─────────────────────────────────────────────────────────────────────────────
aoi_bounds = aoi_native.bounds
pdal_pipeline = [
    str(las_dir / "*.laz"),
    {"type": "filters.crop",
     "bounds": f"[{aoi_bounds[0]},{aoi_bounds[2]}],[{aoi_bounds[1]},{aoi_bounds[3]}]"},
    {"type": "writers.gdal",
     "filename": str(dsm2263),
     "resolution": 0.3048,
     "output_type": "max",
     "data_type": "float32",
     "nodata": -9999,
     "gdaldriver": "GTiff"}
]
with open(pdal_json, "w") as f: json.dump(pdal_pipeline, f, indent=2)
subprocess.run(["pdal", "pipeline", str(pdal_json)], check=True)

# reproject DSM to EPSG:4326
if not dsm4326.exists():
    subprocess.run(["gdalwarp", "-t_srs", CRS_WGS84, "-r", "bilinear",
                    "-dstnodata", "-9999", "-overwrite",
                    dsm2263, dsm4326], check=True)

# ─────────────────────────────────────────────────────────────────────────────
# Summary paths for downstream loader
# ─────────────────────────────────────────────────────────────────────────────
print("\nFinished.\nDEM:", dem4326, "\nDSM:", dsm4326)



--- Downloading DEM/DSM for NYC ---
Querying DEM API: https://elevation.its.ny.gov/arcgis/rest/services/Dem_Indexes/FeatureServer/0/query
Found DEM download URL via API: https://gisdata.ny.gov/elevation/DEM/NYC_TopoBathymetric2017/be_NYC_001.tif


nyc_dem_1ft_2017.zip:  27%|██▋       | 21.0M/77.8M [00:06<00:16, 3.57MiB/s]


KeyboardInterrupt: 